In [56]:
import os
import sys
import json
import pandas as pd

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.11:1.5.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.2 (default, Sep 21 2017 18:29:43)
SparkSession available as 'spark'.


In [3]:
filename = '/Users/ssv/newprolab/labs/lab07/DO_record_per_line.json'

In [4]:
source = sc.textFile(filename)

In [5]:
data = source.map(lambda x: json.loads(x))

In [194]:
dd = data.filter(lambda x: x['lang'] in ['en', 'es', 'ru']).map(lambda x: (x['id'], x['lang'], x['desc']))

In [203]:
rdd_id_words = dd.map(lambda x: (x[0], x[2]))

In [205]:
rdd_id_words.take(5)

[(4,
  'This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They’ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close accounts, and how to prepare key financial reports. All material fo

In [210]:
course_data = rdd_id_words.toDF(schema=['id', 'desc'])

In [218]:
course_data.show(2)

+---+--------------------+
| id|                desc|
+---+--------------------+
|  4|This course intro...|
|  5|This online cours...|
+---+--------------------+
only showing top 2 rows



In [225]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [220]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")

In [221]:
wordsData = tokenizer.transform(course_data)

In [224]:
wordsData.show(3)

+---+--------------------+--------------------+
| id|                desc|               words|
+---+--------------------+--------------------+
|  4|This course intro...|[this, course, in...|
|  5|This online cours...|[this, online, co...|
|  7|We live in a digi...|[we, live, in, a,...|
+---+--------------------+--------------------+
only showing top 3 rows



In [227]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)

In [228]:
featurizedData = hashingTF.transform(wordsData)

In [229]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [240]:
from pyspark.mllib.linalg.distributed import RowMatrix

In [256]:
rdd = rescaledData.select("features").rdd

In [262]:
type(rdd)

pyspark.rdd.RDD

In [259]:
mat = RowMatrix(rdd)

In [263]:
mat.columnSimilarities()

Py4JJavaError: An error occurred while calling o1486.columnSimilarities.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 74.0 failed 1 times, most recent failure: Lost task 0.0 in stage 74.0 (TID 119, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/ssv/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 83, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" %

In [235]:
rescaledData.select("features").show()

+--------------------+
|            features|
+--------------------+
|(10000,[36,42,63,...|
|(10000,[32,222,29...|
|(10000,[493,572,7...|
|(10000,[32,65,115...|
|(10000,[56,91,268...|
|(10000,[1045,1263...|
|(10000,[87,157,57...|
|(10000,[161,164,4...|
|(10000,[26,1072,1...|
|(10000,[63,145,23...|
|(10000,[32,65,77,...|
|(10000,[32,273,30...|
|(10000,[695,1420,...|
|(10000,[307,316,3...|
|(10000,[572,768,8...|
|(10000,[91,273,31...|
|(10000,[148,157,1...|
|(10000,[128,177,2...|
|(10000,[91,332,52...|
|(10000,[115,128,2...|
+--------------------+
only showing top 20 rows



In [237]:
from sklearn.metrics.pairwise import cosine_similarity

In [239]:
cos_m = cosine_similarity(rescaledData.select("features").collect())

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

In [ ]:
import pickle
with open('cos_m.pickle', 'wb') as file:
    pickle.dump(cos_m, file)     
    
# with open('cos_m.pickle', 'rb') as file:
#     cos_m = pickle.load(file)

In [ ]:
from pyspark.mllib.feature import HashingTF, IDF

# Load documents (one per line).
documents = sc.textFile("data/mllib/kmeans_data.txt").map(lambda line: line.split(" "))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

In [81]:
from pyspark.mllib.feature import HashingTF, IDF

In [76]:
tf = HashingTF(10000)

In [85]:
tfVectors = tf.transform(clear_words)

In [ ]:
idf = IDF()
idfModel = idf.fit(tfVectors)
tfIdfVectors = idfModel.transform(tfVectors)